In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import backend as K

# tf.compat.v1.disable_eager_execution()

model = VGG16(weights='imagenet')

In [ ]:
!wget https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.pixoto.com%2Fimages-photography%2Fanimals%2Fother-mammals%2Fscreaming-donky--6470543649800192&psig=AOvVaw0wy7CZeGcqUrSoB7QNZXDr&ust=1736721689346000&source=images&cd=vfe&opi=89978449&ved=2ahUKEwiWxZrM3u6KAxVezQIHHYyFPL0QjRx6BAgAEBk

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

# The local path to our target image
img_path = '/kaggle/input/foximage/fox.jpg'

# `img` is a PIL image of size 224x224
img = image.load_img(img_path, target_size=(224, 224))

# `x` is a float32 Numpy array of shape (224, 224, 3)
x = image.img_to_array(img)

# We add a dimension to transform our array into a "batch"
# of size (1, 224, 224, 3)
x = np.expand_dims(x, axis=0)

# Finally we preprocess the batch
# (this does channel-wise color normalization)
x = preprocess_input(x)

In [ ]:
preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
np.argmax(preds[0])

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Load pre-trained VGG16 model and the image
model = VGG16(weights='imagenet')

# Image preprocessing
img_path = '/kaggle/input/foximage/fox.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Get the predicted class
preds = model.predict(x)
predicted_class = np.argmax(preds[0])
print('Predicted:', decode_predictions(preds, top=3)[0])

# Get the gradients of the predicted class with respect to the output feature map of `block5_conv3`
grad_model = tf.keras.models.Model(
    [model.inputs],  # Model input
    [model.get_layer('block5_conv3').output, model.output]  # Feature map and predictions
)

with tf.GradientTape() as tape:
    # Watch the input
    conv_outputs, predictions = grad_model(x)
    loss = predictions[:, predicted_class]

# Compute gradients
grads = tape.gradient(loss, conv_outputs)

# Compute the channel-wise mean of the gradients
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

# Multiply each channel in the feature map array by "how important this channel is"
conv_outputs = conv_outputs[0]
heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)

# Normalize the heatmap to range [0, 1]
heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

# Display the heatmap
plt.matshow(heatmap)
plt.show()

# Superimpose the heatmap onto the original image
heatmap = cv2.resize(heatmap, (img.size[0], img.size[1])) 
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

superimposed_img = cv2.addWeighted(
    cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR), 0.6, heatmap, 0.4, 0
)

# Save the resulting image
cv2.imwrite('/kaggle/input/foximage/fox.jpg', superimposed_img)

# Show the superimposed image
plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# Ensure heatmap is in float format for division
heatmap = np.maximum(heatmap, 0).astype('float32')
# Normalize the heatmap
heatmap /= np.max(heatmap)
# Display the heatmap
plt.matshow(heatmap)
plt.show()
